In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
from flask import Flask, render_template, redirect

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\laure\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


## NASA Mars News

In [3]:
# NASA Mars News URL of page to be scraped
news_url = "https://redplanetscience.com/"
browser.visit(news_url)
html = browser.html

time.sleep(1)

# Scrape page into Soup
news_soup = bs(html, 'html.parser')

In [4]:
# Identify and return latest news title
results = news_soup.find_all('div', class_="content_title")
news_title = results[1].text
print(f"Latest News Title: {news_title}")

Latest News Title: NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light


In [5]:
# Identify and return latest paragraph text
results = news_soup.find_all('div', class_="article_teaser_body")
news_p = results[0].text
print(f"Paragraph Text: {news_p}")

Paragraph Text: Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. 


## Splinter: JPL Mars - Featured Image

In [6]:
# Setup splinter
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
# JPL Mars Space Images URL of page to be scraped
space_image_url = "https://spaceimages-mars.com/"

# Use Beautiful Soup to parse HTMl via the browser
browser.visit(space_image_url)

# Suspend execution for 4 seconds
time.sleep(4)

html = browser.html

image_soup = bs(html,'html.parser')
image_soup

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [8]:
# Get full image url
image_url1 = image_soup.find('img', class_='headerimage fade-in')['src']
featured_image_url = space_image_url + image_url1
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

## Pandas Scraping: Mars Facts

In [9]:
# Use Pandas to scrape the table on Mars Facts webpage
facts_url = 'https://galaxyfacts-mars.com/'
mars_facts = pd.read_html(facts_url)
facts_df = mars_facts[0]
facts_df.columns = ["Properties", "Mars", "Earth"]
facts_df = facts_df.set_index("Properties")
facts_df

,Mars,Earth
Properties,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
# Convert data to a HTML table string. Clean up by removing /n
mars_facts = facts_df.to_html()
mars_facts.replace('\n','')
print(mars_facts)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Properties</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

In [11]:
# Setup splinter
browser = Browser('chrome', **executable_path, headless=False)

In [12]:
# URL of page to be scraped
hemispheres_url = 'https://marshemispheres.com/'
browser.visit(hemispheres_url)
html = browser.html
astro_soup = bs(html, 'html.parser')

In [13]:
# Find <h3> titles containing 'Hemisphere' 
titles = astro_soup.find_all("h3")
for title in titles:
    browser.links.find_by_partial_text("Hemisphere")
print(titles)

[<h3>Cerberus Hemisphere Enhanced</h3>, <h3>Schiaparelli Hemisphere Enhanced</h3>, <h3>Syrtis Major Hemisphere Enhanced</h3>, <h3>Valles Marineris Hemisphere Enhanced</h3>, <h3>Back</h3>]


In [14]:
# Extract Results where class = collapsible results
mars_hems = astro_soup.find("div", class_="collapsible results")
results = mars_hems.find_all('div', class_="item")

# Create empty list for images
hemisphere_images = []

# Loop through results
for result in results:
    # Use bs to find title
    title = astro_soup.find('h3').text.strip()
    # Use bs to find and create image link
    href = result.find('a')['href']
    browser.visit(hemispheres_url + href)
    html = browser.html
    src_soup = bs(html, 'html.parser')
    image_link = src_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']
    full_path = hemispheres_url + image_url
     # Append title and image URL to dict
    hemisphere_images.append({"title":title,"img_url":full_path})

# Results
hemisphere_images

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [15]:
browser.quit() 

## Save Scraped Data

In [16]:
# Create dictionary for all Mars info from above sources
mars_dict = {
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "mars_facts":str(mars_facts),
    "hemisphere_images":hemisphere_images
}
print(mars_dict)

{'news_title': "NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light", 'news_p': 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. ', 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars1.jpg', 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Properties</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Di